FLOWER CLASSIFICATION

In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

Assigning Unique Labels

In [4]:
data = pd.read_csv("C:/Users/hp/Desktop/pandas/iris.csv")
data = data.drop(["Id"], axis=1)

un_labels = data["Species"].unique()
# for i in data["Species"].unique():
#     print(sum(data["Species"]==i))

data = shuffle(data)
labels = data.pop("Species")
labels = np.array(labels)

data = ss().fit_transform(data)
print(un_labels)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [11]:
ohe_labels = np.zeros((len(labels), len(un_labels)))

for i in range(len(un_labels)):
    x = np.where(labels == un_labels[i])
    ohe_labels[x,i] = 1

ohe_labels[:10]

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [12]:
features = torch.tensor(np.array(data), dtype=torch.float32)
labels = torch.tensor(ohe_labels , dtype=torch.float32)

In [13]:
model = torch.nn.Sequential(
    torch.nn.Linear(4,4),
    torch.nn.ReLU(),
    torch.nn.Linear(4,3),
    torch.nn.Softmax(dim=-1)
)

preds = model(features)
print(preds[:5])

tensor([[0.3481, 0.3673, 0.2846],
        [0.3367, 0.2792, 0.3841],
        [0.3272, 0.2342, 0.4387],
        [0.3294, 0.2423, 0.4284],
        [0.3458, 0.3606, 0.2935]], grad_fn=<SliceBackward0>)


Predicting the Loss

In [14]:
loss_fn = torch.nn.BCELoss()

loss = loss_fn(preds, labels)
print(loss)

tensor(0.6970, grad_fn=<BinaryCrossEntropyBackward0>)


In [15]:
from torch import tensor


opt = torch.optim.SGD(model.parameters(), lr=0.01)
epochs = 10000

for epoch in range(epochs):
    preds = model(features)
    loss = loss_fn(preds, labels)

    loss.backward()

    opt.step()
    opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            level = 0
            for i in range(len(preds)):
                preds_index = torch.where(preds[i] == preds[i].max())
                if (ohe_labels[i][preds_index]) ==1:
                    level += 1

            print(f"Loss : {loss}")
            print(f"Accuracy : {round(level * 100/ len(preds), 1)}%")

Loss : 0.3071741759777069
Accuracy : 73.3%
Loss : 0.1987932175397873
Accuracy : 90.7%
Loss : 0.13984891772270203
Accuracy : 94.7%
Loss : 0.10282380133867264
Accuracy : 96.0%
Loss : 0.07887386530637741
Accuracy : 97.3%
Loss : 0.06418417394161224
Accuracy : 97.3%
Loss : 0.055028609931468964
Accuracy : 97.3%
Loss : 0.0490461066365242
Accuracy : 97.3%
Loss : 0.04490124061703682
Accuracy : 97.3%
Loss : 0.04190182313323021
Accuracy : 98.0%
